In [1]:

import logging
import pandas as pd

import feedparser
from datetime import datetime
import dateutil
from pytz import timezone

d = feedparser.parse('https://weather.gc.ca/rss/city/ab-52_e.xml')
##print(d.feed.title)
len(d['items'])

fcstutc = dateutil.parser.parse(d['items'][2]['published'])
##print(fcstutc)

fcstupd = fcstutc.astimezone(timezone('America/Edmonton'))

popDF =  pd.DataFrame(columns=['Day', 'DayNight', 'POP','Date'])

print(fcstupd,'\n',popDF)

2019-07-01 16:17:00-06:00 
 Empty DataFrame
Columns: [Day, DayNight, POP, Date]
Index: []


In [2]:


for i in range(0,14):
    e = d['items'][i]
##    print("\nNew Day ", e['title'],"\n")
    fcst=e['title'].split(':')
##    print(len(fcst),'\n')
            
    
    dayNight = fcst[0].split(' ')
    
##    print(len(dayNight))
    
    theDay = dayNight[0]
    if len(dayNight)>1:
        if (dayNight[1] == 'night'):
            timeofDay = 'night'
    else:
        timeofDay = 'day'
    
    
#     print(theDay, timeofDay)
    
    if len(fcst)>1:
##        print('second part ',fcst[1])
        if fcst[1].find('POP') > 0:
            pop = fcst[1].split('POP')
##            print('\nPOP ',pop[1])
            popDF = popDF.append({'Day': theDay, 'DayNight': timeofDay, 'POP': pop[1]}, ignore_index=True)

        
print('\nPOP days\n',popDF)


POP days
          Day DayNight   POP Date
0     Monday    night   60%  NaN
1    Tuesday      day   30%  NaN
2  Wednesday    night   40%  NaN
3   Thursday      day   40%  NaN
4   Thursday    night   60%  NaN
5     Friday    night   40%  NaN
6   Saturday      day   60%  NaN
7   Saturday    night   60%  NaN
8     Sunday      day   60%  NaN


In [3]:


from datetime import timedelta
def get_next_monday():
    date0 = datetime.now()
#     date0 = datetime.date(year, month, day)
    next_monday = date0 + timedelta(7 - date0.weekday() or 7)
    return next_monday

##print (get_next_monday())

import calendar
import pandas as pd

fcstDays = pd.DataFrame(columns=['Day', 'Date'])

for dd in range(0,7):
    
    dayN = datetime.now() + timedelta(days=dd)
    dayName = calendar.day_name[dayN.weekday()]
    dayDt = dayN.date()
    
    fcstDays = fcstDays.append({'Day': dayName, 'Date': dayDt}, ignore_index=True)

##fcstDays

fcstwk = pd.merge(popDF, fcstDays, on='Day')

fcstwk.sort_values('Date_y')
fcstwk.Date_x=fcstupd

fcstwk = fcstwk.rename(columns={'Date_x':'Forecast Date','Date_y':'Date Forecasted'})

##print(fcstwk)

fcstHist = pd.read_pickle('/home/mark/Documents/irrigation/weather/FcstHistory.pkl')

# fcstwk.loc[fcstwk.DayNight == 'day', fcstwk['Date Forecasted']] = pd.to_datetime(fcstwk['Date Forecasted'])+ pd.Timedelta(11, unit='h')

for Index, row in fcstwk.iterrows():
    if row.DayNight == 'day':
        zoink = pd.to_datetime(row['Date Forecasted'])+ pd.Timedelta(11, unit='h')
#         print(zoink, 'index ',row.index, Index)
    else:
        zoink = pd.to_datetime(row['Date Forecasted'])+ pd.Timedelta(18, unit='h')
#         print(zoink, 'index ',Index)
        
    fcstwk.at[Index, 'Date Forecasted'] = zoink
    
fcstwk.POP = pd.to_numeric(fcstwk.POP.str.strip('%'))/100

# fcstwk = fcstwk[['Date Forecasted','Forecast Date','POP']]

# fcstHist = fcstHist.append(fcstwk, ignore_index=True)

# fcstHist = fcstHist.drop_duplicates()


fcstHist.to_pickle('/home/mark/Documents/irrigation/weather/FcstHistory.pkl')

pfcst = fcstwk.sort_values(by=['Date Forecasted','DayNight','Forecast Date'],ascending=[True,True,False])



print(pfcst[['Day','Forecast Date','Date Forecasted','POP']])

# fcstwk



         Day             Forecast Date      Date Forecasted  POP
0     Monday 2019-07-01 16:17:00-06:00  2019-07-01 18:00:00  0.6
1    Tuesday 2019-07-01 16:17:00-06:00  2019-07-02 11:00:00  0.3
2  Wednesday 2019-07-01 16:17:00-06:00  2019-07-03 18:00:00  0.4
3   Thursday 2019-07-01 16:17:00-06:00  2019-07-04 11:00:00  0.4
4   Thursday 2019-07-01 16:17:00-06:00  2019-07-04 18:00:00  0.6
5     Friday 2019-07-01 16:17:00-06:00  2019-07-05 18:00:00  0.4
6   Saturday 2019-07-01 16:17:00-06:00  2019-07-06 11:00:00  0.6
7   Saturday 2019-07-01 16:17:00-06:00  2019-07-06 18:00:00  0.6
8     Sunday 2019-07-01 16:17:00-06:00  2019-07-07 11:00:00  0.6


In [25]:
fcstwk.dtypes


Day                                          object
DayNight                                     object
POP                                          object
Forecast Date      datetime64[ns, America/Edmonton]
Date Forecasted                              object
dtype: object

In [37]:
snurg = datetime.now()
xfr = pd.Timedelta(11, unit='h')
# zpla = snidge + xfr
zpla = snidge + pd.Timedelta(11, unit='h')
zpla

0   2019-06-30 11:00:00
1   2019-07-01 11:00:00
2   2019-07-03 11:00:00
3   2019-07-04 11:00:00
4   2019-07-05 11:00:00
5   2019-07-05 11:00:00
6   2019-07-06 11:00:00
Name: Date Forecasted, dtype: datetime64[ns]